# Cleaning Many Datasets

In [1]:
import dask, dask.dataframe as dd
import matplotlib.pyplot as plt
import pandas as pd
import re, csv, os
import numpy as np

pd.set_option('display.max_columns', None)
csv.field_size_limit(10000000)

%matplotlib inline

## What to do before running these cells

Add the path to your files to the variable **path** below.

For **partitions_out** below think about how many GB you will be cleaning and how many files you will like to have at the end of the cleaning process. A good rule of thumb is to split large files into manageable chunks of 300 to 600 MB for analysis. If you would like to follow this approach, figure out how much data you will be cleaning in MB terms (1 GB = 1000 MB) and divide it by the size in MB terms that you would like your final files to have. For example, 3GB (or 3,000MB) divided by 300MB would amount to 10 partitions.

For **partitions_in**, do something somewhat similar than with **partitions_out** but to a much larger scale. If you are cleaning 100 GB of data make about 1000 partitions so that dask can clean faster with very manageable chunks.

In [2]:
path = '/Volumes/LaCie SSD/bgdata/data_19/some_data/new_test'
partitions_in = 10
partitions_out = 5

The following are the variables which I've determined the most useful. Feel free to add or subtract from them before running the cells below. No need to update the `dtypes` dictionary below as it contains all the variables in the BG dataset.

In [6]:
best_list = ['JobID', 'CleanJobTitle', 'CanonCity', 'CanonState', 'JobDate', 'JobText', 'Source', 'CanonEmployer',
             'Latitude', 'Longitude', 'CanonIntermediary', 'CanonJobTitle', 'CanonCounty', 'DivisionCode', 'MSA', 'LMA',
             'InternshipFlag', 'ConsolidatedONET', 'CanonSkillClusters', 'CanonSkills', 'IsDuplicate', 'CanonMinimumDegree', 
             'CanonRequiredDegrees', 'CIPCode', 'MinExperience', 'ConsolidatedInferredNAICS', 'BGTOcc', 'MaxAnnualSalary', 'MaxExperience',
             'MaxHourlySalary', 'MinAnnualSalary', 'MinHourlySalary', 'YearsOfExperience', 'CanonJobHours', 'CanonJobType',
             'CanonPostalCode', 'CanonYearsOfExperienceCanonLevel', 'CanonYearsOfExperienceLevel', 'ConsolidatedTitle', 'MaxExperience',
             'Language', 'BGTSubOcc', 'ConsolidatedDegreeLevels', 'MaxDegreeLevel', 'MinDegreeLevel', 'CanonMaximumDegree']

Because the data is very messy and dask can't infer correctly all of the variables' data types without taking away the gain of parallelizing the computations, we will import every var with the data type as a string.

In [4]:
dtypes={'JobID': np.str, 'CleanJobTitle': np.str, 'JobDomain': np.str, 'CanonCity': np.str, 'CanonCountry': np.str, 'CanonState': np.str, 
        'JobText': np.str, 'JobURL': np.str, 'PostingHTML': np.str, 'Source': np.str, 'JobReferenceID': np.str, 'Email': np.str, 
        'CanonEmployer': np.str, 'Latitude': np.str, 'Longitude': np.str, 'CanonIntermediary': np.str, 'Telephone': np.str, 'CanonJobTitle': np.str, 
        'CanonCounty': np.str, 'DivisionCode': np.str, 'MSA': np.str, 'LMA': np.str, 'InternshipFlag': np.str, 'ConsolidatedONET': np.str, 'CanonCertification': np.str, 
        'CanonSkillClusters': np.str, 'CanonSkills': np.str, 'IsDuplicate': np.str, 'IsDuplicateOf': np.str, 'CanonMaximumDegree': np.str, 'CanonMinimumDegree': np.str, 
        'CanonOtherDegrees': np.str, 'CanonPreferredDegrees': np.str, 'CanonRequiredDegrees': np.str, 'CIPCode': np.str, 'StandardMajor': np.str, 
        'MaxExperience': np.str, 'MinExperience': np.str, 'ConsolidatedInferredNAICS': np.str, 'BGTOcc': np.str, 'MaxAnnualSalary': np.str, 'MaxHourlySalary': np.str, 
        'MinAnnualSalary': np.str, 'MinHourlySalary': np.str, 'YearsOfExperience': np.str, 'CanonJobHours': np.str, 'CanonJobType': np.str, 'CanonPostalCode': np.str, 
        'CanonYearsOfExperienceCanonLevel': np.str, 'CanonYearsOfExperienceLevel': np.str, 'ConsolidatedTitle': np.str, 'Language': np.str, 'BGTSubOcc': np.str, 'JobDate': np.str,
        'ConsolidatedDegreeLevels': np.str, 'MaxDegreeLevel': np.str, 'MinDegreeLevel': np.str,
                       }

Notice the wildcard in the `os.path.join()` call of your dask dataframe `read_csv` function. That tells Dask to grab all of the files that end with `'.csv'` inside your folder. You can make it more specific by adding more characters before and after the star. For example, `'data_0*.csv'` will grab all CSV files in your folder that start with `data_0` and end with `.csv`.

Also notice the we pass in the list of variables and the the dictionary of data types. We also tell dask to assume that there will be missing data with the parameter `assume_missing`. Error bad lines will print the bad lines that dask skips for us.

Make sure to add a few letters from the start of your files.

Now run everything and wait. :)

In [7]:
ddf = dd.read_csv(os.path.join(path, 'da*.csv'), 
                 engine='python', 
                 dtype=dtypes,
                 assume_missing=True,
                 error_bad_lines=False,
                 blocksize=None,
                 usecols=best_list,
                )
ddf

,JobID,CleanJobTitle,CanonCity,CanonState,JobDate,JobText,Source,CanonEmployer,Latitude,Longitude,CanonIntermediary,CanonJobTitle,CanonCounty,DivisionCode,MSA,LMA,InternshipFlag,ConsolidatedONET,CanonSkillClusters,CanonSkills,IsDuplicate,CanonMaximumDegree,CanonMinimumDegree,CanonRequiredDegrees,CIPCode,MaxExperience,MinExperience,ConsolidatedInferredNAICS,BGTOcc,MaxAnnualSalary,MaxHourlySalary,MinAnnualSalary,MinHourlySalary,YearsOfExperience,CanonJobHours,CanonJobType,CanonPostalCode,CanonYearsOfExperienceCanonLevel,CanonYearsOfExperienceLevel,ConsolidatedTitle,Language,BGTSubOcc,ConsolidatedDegreeLevels,MaxDegreeLevel,MinDegreeLevel
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [8]:
# here is where we repartition our data
ddf00 = ddf.repartition(npartitions=partitions_in)
ddf00.npartitions

10

In [ ]:
# %%time

# ddf00.head()

In [30]:
# %%time

# ddf00.tail()

In [9]:
# There are missing company names that map to a recruiting agency and because of this
# we will identify those observations and fill in the missing valyes in the CanonEmployer
# var with "Recruitment Agency"
EmployerCondition = ((ddf00['CanonEmployer'].isnull()) & (ddf00['CanonIntermediary'].notnull()))
EmployerClean = ddf00['CanonEmployer'].where(~EmployerCondition, 'Recruitment Agency')

# we will then drop the original variable and add the new one to the dataset using the following methods
ddf_clean0 = ddf00.drop('CanonEmployer', axis=1)
ddf_clean01 = ddf_clean0.assign(EmployerClean=EmployerClean)

In [ ]:
### Note: The following cell will take a while!

%%time

# We have a lot of missing values in this dataset so let's start by calculating those
# as a percentage of all of the samples in our datasets
# missing_count = ((ddf_clean01.isna().sum() / ddf_clean01.index.size) * 100)
# missing_count_pct = missing_count.compute()
# missing_count_pct

# we will now drop the columns with 60% or more missing values
# cols_to_drop = ['CanonIntermediary', 'DivisionCode']

# Since the rows above have more than 60, 70 and 80% of missing values,
# we will be getting rid of them with the drop command
# ddf_clean1 = ddf_clean01.drop(cols_to_drop, axis=1)

# since english must be the most common language for the majority of positions in 
# the USA, we will fill in missing values in that colunm with the en value in the Language var
# ddf_clean2 = ddf_clean1.fillna({})

In [13]:
# here we will get rid of the rows in columns with missing values 
# between 1 and 10%
rows_to_drop = ['CleanJobTitle', 'CanonCity', 'CanonState', 'Source', 'Latitude', 'Longitude', 'CanonCounty',
                'MSA', 'LMA', 'ConsolidatedONET', 'BGTOcc', 'CanonPostalCode', 'ConsolidatedTitle',
                'BGTSubOcc', 'EmployerClean', 'JobText', 'JobID', 'JobDate']

In [14]:
# here is the code to drop them
ddf_clean02 = ddf_clean01.dropna(subset=rows_to_drop)

In [15]:
# we will assign the word "Unknown" the remaining columns with missing values
# The nice thing about python and many other languages is that we can read the data
# and tell it to reassign np.nan to observations containing the word "Unknown"
# remaining_cols_to_clean = list(missing_count_pct[(missing_count_pct >= 10) & (missing_count_pct < 60)].index)
unknown_default_dict = {'CanonJobTitle': 'Unknown', 'CanonMinimumDegree': 'Unknown', 'CanonRequiredDegrees': 'Unknown', 'InternshipFlag': 'Unknown',
                        'MinExperience': 'Unknown', 'ConsolidatedInferredNAICS': 'Unknown', 'YearsOfExperience': 'Unknown', 'Language': 'en', 'MaxExperience': 'Unknown',
                        'CanonJobHours': 'Unknown', 'CanonJobType': 'Unknown', 'CanonYearsOfExperienceCanonLevel': 'Unknown', 'CanonSkillClusters':'Unknown',
                        'CanonYearsOfExperienceLevel': 'Unknown', 'ConsolidatedDegreeLevels': 'Unknown', 'MinDegreeLevel': 'Unknown', 'IsDuplicate': 'Unknown',
                        'CIPCode': 'Unknown', 'MaxAnnualSalary': -9999, 'MaxHourlySalary': -9999, 'MinAnnualSalary': -9999, 'CanonSkills': 'Unknown',
                        'MinHourlySalary': -9999, 'MaxDegreeLevel': 'Unknown', 'CanonIntermediary': 'Unknown', 'DivisionCode': 'Unknown', 'CanonMaximumDegree': 'Unknown'}

In [16]:
# here we fill in those missing values
ddf_clean03 = ddf_clean02.fillna(unknown_default_dict)

In [17]:
# The JobText var is not formatted correctly so we will first clean it
# and create a new variable called clean_text
clean_text = ddf_clean03.loc[:, 'JobText'].str.strip()
# .apply(lambda x: ' '.join(list(filter(None, x.split()))), meta=np.str)

# we will then drop the old JobText var
ddf_clean04 = ddf_clean03.drop('JobText', axis=1)

# Here we reassign the cleaned var back into the dataset
ddf_clean05 = ddf_clean04.assign(clean_text=clean_text)

# we will now filter out job descriptions that are not written in english
english_condition = ddf_clean05['Language'].isin(['en'])
ddf_clean06 = ddf_clean05[english_condition]

# let's filter out duplicate jobs and then drop that column
no_duplicates = (ddf_clean06['IsDuplicate'] == "FALSE")
ddf_clean07 = ddf_clean06[no_duplicates]
ddf_clean08 = ddf_clean07.drop(['IsDuplicate', 'Language'], axis=1)

### Note: The following cell will take a while!

In [18]:
%%time

# using the same folder in your path, we will create a new one for the cleaned data
# and save our new files there
if not os.path.exists(os.path.join(path, 'clean2')):
    os.makedirs(os.path.join(path, 'clean2'))
    

# the following lines of code will take the last dataset, repartition it,
# and save it to the desired location. Notice the wildcard "*" below. That is
# the spot Dask will use to number your files starting from 0
(ddf_clean08
 .repartition(partition_size='500MB'
#               npartitions=partitions_out
             )
 .to_csv(os.path.join(path, 'clean2/', 'data_cleaned_*.csv'), index=False)
 )

CPU times: user 3min 16s, sys: 9min 55s, total: 13min 11s
Wall time: 31min 37s


['/Volumes/LaCie SSD/bgdata/data_19/some_data/new_test/clean2/data_cleaned_0.csv',
 '/Volumes/LaCie SSD/bgdata/data_19/some_data/new_test/clean2/data_cleaned_1.csv',
 '/Volumes/LaCie SSD/bgdata/data_19/some_data/new_test/clean2/data_cleaned_2.csv',
 '/Volumes/LaCie SSD/bgdata/data_19/some_data/new_test/clean2/data_cleaned_3.csv',
 '/Volumes/LaCie SSD/bgdata/data_19/some_data/new_test/clean2/data_cleaned_4.csv']